In [3]:
import pandas as pd
from pandas import DataFrame
import random as rm
import numpy as np
import time
import networkx as nx
import pickle
import datetime
from datetime import timedelta
import scipy.stats
import matplotlib.pyplot as plt

In [2]:
#Process raw timetable data into a more manageable dataframe
def Process_raw_timetable(Timetablepath,Timetablefilename):
    #Read in timetable data
    data_folder = Timetablepath
    filename = data_folder+ Timetablefilename
    data = pd.read_csv(filename, header=None, names=['tag'])
    data['Msg_type'] = data['tag'].str.slice(0,2)
    data = data[data['Msg_type']!='AA'].reset_index(drop=True)
    #Process basic schedule data type
    BSdata= data[data['Msg_type']=='BS'].copy()
    BSdata['TransactionType'] = BSdata['tag'].str.slice(2,3)
    BSdata['TrainUID'] = BSdata['tag'].str.slice(3,9)
    BSdata['Datefrom'] = BSdata['tag'].str.slice(9,15)
    BSdata['Dateto'] = BSdata['tag'].str.slice(15,21)
    BSdata['Daysrun'] = BSdata['tag'].str.slice(21,28)
    BSdata['Bankhol'] = BSdata['tag'].str.slice(28,29)
    BSdata['Trainstatus'] = BSdata['tag'].str.slice(29,30)
    BSdata['Traincategory'] = BSdata['tag'].str.slice(30,32)
    BSdata['Trainidentity'] = BSdata['tag'].str.slice(32,36)
    BSdata['Headcode'] = BSdata['tag'].str.slice(36,40)
    BSdata['Servicecode'] = BSdata['tag'].str.slice(41,49)
    BSdata['Speed'] = BSdata['tag'].str.slice(57,60)
    BSdata['Operatechar'] = BSdata['tag'].str.slice(60,66)
    BSdata['STP']=BSdata['tag'].str.slice(79,80)
    del BSdata['tag']
    BSdata=BSdata.reset_index()
    STPC=BSdata[BSdata['STP']=='C'].reset_index(drop=True)
    for i in range(0,len(BSdata)-1):
        if BSdata['index'][i]==BSdata['index'][i+1]-1:
            BSdata=BSdata.drop([i])
    BSdata=BSdata.reset_index(drop=True)
    #Process train origin data type
    LOdata= data[data['Msg_type']=='LO'].copy()
    LOdata['Location'] = LOdata['tag'].str.slice(2,10)
    LOdata['SchedArr'] = LOdata['tag'].str.slice(50,51)
    LOdata['SchedDept'] = LOdata['tag'].str.slice(10,15)
    LOdata['SchedPass'] = LOdata['tag'].str.slice(50,51)
    LOdata['PubArr'] = LOdata['tag'].str.slice(50,51)
    LOdata['PubDept'] = LOdata['tag'].str.slice(15,19)
    LOdata['Platform'] = LOdata['tag'].str.slice(19,22)
    LOdata['Line'] = LOdata['tag'].str.slice(22,25)
    LOdata['Path'] = LOdata['tag'].str.slice(50,51)
    LOdata['Activity'] = LOdata['tag'].str.slice(29,41)
    LOdata['Engallow'] = LOdata['tag'].str.slice(25,27)
    LOdata['Pathallow'] = LOdata['tag'].str.slice(27,29)
    LOdata['Perfallow'] = LOdata['tag'].str.slice(41,43)
    del LOdata['tag']
    #Process intermediate train movements data type
    LIdata= data[data['Msg_type']=='LI'].copy()
    LIdata['Location'] = LIdata['tag'].str.slice(2,10)
    LIdata['SchedArr'] = LIdata['tag'].str.slice(10,15)
    LIdata['SchedDept'] = LIdata['tag'].str.slice(15,20)
    LIdata['SchedPass'] = LIdata['tag'].str.slice(20,25)
    LIdata['PubArr'] = LIdata['tag'].str.slice(25,29)
    LIdata['PubDept'] = LIdata['tag'].str.slice(29,33)
    LIdata['Platform'] = LIdata['tag'].str.slice(33,36)
    LIdata['Line'] = LIdata['tag'].str.slice(36,39)
    LIdata['Path'] = LIdata['tag'].str.slice(39,42)
    LIdata['Activity'] = LIdata['tag'].str.slice(40,54)
    LIdata['Engallow'] = LIdata['tag'].str.slice(54,56)
    LIdata['Pathallow'] = LIdata['tag'].str.slice(56,58)
    LIdata['Perfallow'] = LIdata['tag'].str.slice(58,60)
    del LIdata['tag']
    #Process train terminal movements data type
    LTdata= data[data['Msg_type']=='LT'].copy()
    LTdata['Location'] = LTdata['tag'].str.slice(2,10)
    LTdata['SchedArr'] = LTdata['tag'].str.slice(10,15)
    LTdata['SchedDept'] = LTdata['tag'].str.slice(50,51)
    LTdata['SchedPass'] = LTdata['tag'].str.slice(50,51)
    LTdata['PubArr'] = LTdata['tag'].str.slice(15,19)
    LTdata['PubDept'] = LTdata['tag'].str.slice(50,51)
    LTdata['Platform'] = LTdata['tag'].str.slice(19,22)
    LTdata['Line'] = LTdata['tag'].str.slice(50,51)
    LTdata['Path'] = LTdata['tag'].str.slice(22,25)
    LTdata['Activity'] = LTdata['tag'].str.slice(25,37)
    LTdata['Engallow'] = LTdata['tag'].str.slice(50,51)
    LTdata['Pathallow'] = LTdata['tag'].str.slice(50,51)
    LTdata['Perfallow'] = LTdata['tag'].str.slice(50,51)
    del LTdata['tag']
    #Recombine these message types to get to the timetable
    Timetabledata=LOdata.append(LIdata)
    Timetabledata=Timetabledata.append(LTdata)
    Timetabledata=Timetabledata.sort_index()
    Timetabledata=Timetabledata.reset_index()
    del Timetabledata['index']
    #Add additional information from the basic schedule messages
    k=0
    Timetabledata['TrainUID']=Timetabledata['Location']
    Timetabledata['Headcode']=Timetabledata['Location']
    Timetabledata['Daysrun']=Timetabledata['Location']
    Timetabledata['Datesrun']=Timetabledata['Location']
    for i in range(0,len(Timetabledata)):
        Timetabledata.at[i,'TrainUID']=BSdata['TrainUID'][k]
        Timetabledata.at[i,'Headcode']=BSdata['Trainidentity'][k]
        Timetabledata.at[i,'Daysrun']=BSdata['Daysrun'][k]
        Timetabledata.at[i,'Datesrun']=BSdata['Datefrom'][k]+BSdata['Dateto'][k]
        Timetabledata.at[i,'STP']=BSdata['STP'][k]
        if Timetabledata['Msg_type'][i]=='LT':
            k=k+1
    Timetabledata['Datefrom']=pd.to_datetime('20'+Timetabledata['Datesrun'].str.slice(0,6),format='%Y%m%d')
    Timetabledata['Dateto']=pd.to_datetime('20'+Timetabledata['Datesrun'].str.slice(6,12),format='%Y%m%d')
    del Timetabledata['Datesrun']
    return Timetabledata,STPC

In [ ]:
Timetabledata,STPC=Process_raw_timetable("/home/mathsys1/Documents/Train_Data/Data_from_Ian/","POINTA20190930OU.LPF")
Timetabledata

In [ ]:
Timetabledata[Timetabledata['Location']=='PITSEA  ']

In [ ]:
del STPC['Bankhol']
del STPC['Trainstatus']
del STPC['Traincategory']
del STPC['Trainidentity']
del STPC['Headcode']
del STPC['Servicecode']
del STPC['Speed']
del STPC['Operatechar']
STPC

In [ ]:
#pickle_out=open("Alt2019-09-02Timetable.pickle","wb")
#pickle.dump(Timetabledata, pickle_out)
#pickle_out=open("2019-09-01STP.pickle","wb")
#pickle.dump(Timetabledata['STP'], pickle_out)

In [ ]:
pickle_out=open("2019-09-30STPC.pickle","wb")
pickle.dump(STPC, pickle_out)

In [ ]:
BSdata

In [3]:
#Read and process the STANOX-to-berth data effectively turning it into
# TIPLOC-to-berth data
def STANOXberth_processing(STANOXberthpath,STANOXberthfilename,TIPLOCSTANOXpath,TIPLOCSTANOXfilename):
    #Read in the data
    data_folder = STANOXberthpath
    filename = data_folder+STANOXberthfilename
    STANOXberth=pd.read_csv(filename)
    #Also requires the STANOX-TIPLOC data
    data_folder = TIPLOCSTANOXpath
    filename = data_folder+TIPLOCSTANOXfilename
    STANOXTIPLOC=pd.read_csv(filename)
    #Append TIPLOCs to the STANOXberth dataframe
    STANberth=STANOXberth.copy()
    STANberth['TIPLOC']=str(STANberth['STANOX'])
    for i in range(0,len(STANberth['STANOX'])):
        Interim=STANOXTIPLOC[STANOXTIPLOC['STANOX']==str(STANberth['STANOX'][i])].reset_index()
        if len(Interim)>0:
            STANberth.at[i,'TIPLOC']=Interim['TIPLOC'][0]
        else:
            STANberth.at[i,'TIPLOC']='NaN'
    
    #Removes all ofsets except the standard 'B' type offset
    STANberth=STANberth[STANberth['STEPTYPE']=='B'].reset_index(drop=True)
    return STANberth

In [ ]:
STANberth=STANOXberth_processing("/home/mathsys1/Documents/","berth_step_mapping_20191120.csv","/home/mathsys1/Documents/","TIPLOC-STANOX.csv")
STANberth

In [4]:
#Adding trigger berths and offsets to the dataframe of timetable data
def Process_cleaned_timetable(Timetablepath,Timetablefilename,STANOXberthpath,STANOXberthfilename,TIPLOCSTANOXpath,TIPLOCSTANOXfilename):
    #Call previous functions to have prerequisites 
    Timetabledata,STPC=Process_raw_timetable(Timetablepath,Timetablefilename)
    STANberth=STANOXberth_processing(STANOXberthpath,STANOXberthfilename,TIPLOCSTANOXpath,TIPLOCSTANOXfilename)
    for i in range(0,len(Timetabledata)):
        Timetabledata.at[i,'Location']=Timetabledata['Location'][i].strip()
        for j in range(0,5):
            Timetabledata.at[i,'Location']=Timetabledata['Location'][i].rstrip(str(j))
    #Add berths and relevant offsets to the Timetabledata dataframe
    Timetabledata['Berth']=np.array
    Timetabledata['Offset']=np.array
    my_list=[]
    my_other_list=[]
    for i in range(0,len(Timetabledata['Location'])):
        prep=STANberth[STANberth['TIPLOC']==Timetabledata['Location'][i]].reset_index()
        if len(prep['FROMBERTH'])>0:
            for j in range(0,len(prep['FROMBERTH'])):
                tupl=list([prep['FROMBERTH'][j],prep['TOBERTH'][j]])
                my_list=my_list+[tupl]
                my_other_list=my_other_list+[prep['STOPPINGOFFSET'][j]]
            Timetabledata.at[i,'Berth']=my_list
            Timetabledata.at[i,'Offset']=my_other_list
            my_list=[]
            my_other_list=[]
        else:
            Timetabledata.at[i,'Berth']='NaN'
            Timetabledata.at[i,'Offset']='NaN'
    return Timetabledata

In [ ]:
Timetabledata=Process_cleaned_timetable("/home/mathsys1/Documents/Train_Data/Data_from_Ian/","POINTA20190930OU.LPF","/home/mathsys1/Documents/","berth_step_mapping_20191120.csv","/home/mathsys1/Documents/","TIPLOC-STANOX.csv")
Timetabledata

In [ ]:
filename="/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle"
pickle_in=open(filename,"rb")
STANberth=pickle.load(pickle_in)
STANberth1=STANberth[STANberth['TD_ID']=='UR'].reset_index(drop=True)
#STANberth2=STANberth[STANberth['TD_ID']=='U2'].reset_index(drop=True)
a=STANberth1['TIPLOC'].unique()
#b=STANberth2['TIPLOC'].unique()
#a=np.append(a,b)
a

In [ ]:
for i in range(0,len(Timetabledata)):
    if np.isin(Timetabledata['Location'][i][0:7],a)==True:
        Timetabledata.at[i,'Path']='UR'
    #if np.isin(Timetabledata['Location'][i][0:7],b)==True:
    #    Timetabledata.at[i,'Path']='U2'
Timetabledata

In [ ]:
Timetabledata[Timetabledata['Location']=='PITSEA']

In [ ]:
Your=Timetabledata[Timetabledata['Path']=='UR'].reset_index(drop=True)
#You2=Timetabledata[Timetabledata['Path']=='U2'].reset_index(drop=True)
RelData=Your.copy()#append(You2).reset_index(drop=True)
RelData

In [ ]:
pickle_out=open("Alt2019-09-30Timetable.pickle","wb")
pickle.dump(RelData, pickle_out)

In [ ]:
filename="/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Alt2019-09-01Timetable.pickle"
pickle_in=open(filename,"rb")
Time=pickle.load(pickle_in)
Time

In [ ]:
Time[Time['Location']=='PITSEA']

In [ ]:
#RelData['Daysrun']=RelData['STP']+RelData['Daysrun']
#Export=RelData.to_csv("/home/mathsys1/Documents/Alt2019-09-01Timetable.csv",index=None,header=True)

In [5]:
#Selects a day of data based on inputs, calls previous functions
#to get data so takes a while to run
def Day_iso(Dayinput,Timetablepath,Timetablefilename,STANOXberthpath,STANOXberthfilename,TIPLOCSTANOXpath,TIPLOCSTANOXfilename):
    Timetabledata=Process_cleaned_timetable(Timetablepath,Timetablefilename,STANOXberthpath,STANOXberthfilename,TIPLOCSTANOXpath,TIPLOCSTANOXfilename)
    WedTimetable=Timetabledata.copy()
    #Dayinput format is 'YYYY-MM-DD 00:00:00'
    Date=pd.Timestamp(Dayinput, freq='D')
    WedTimetable=WedTimetable[WedTimetable['Date']==Date]
    WedTimetable=WedTimetable.reset_index(drop=True)
    return WedTimetable

In [ ]:
Selectdayschedule=Day_iso("2019-09-01 00:00:00","/home/mathsys1/Documents/Train_Data/Data_from_Ian/","POINTA20190901LF.csv","/home/mathsys1/Documents/","berth_step_mapping_20191120.csv","/home/mathsys1/Documents/","TIPLOC-STANOX.csv")
Selectdayschedule

In [ ]:
pickle_out=open("New2019-09-01Timetable.pickle","wb")
pickle.dump(Timetabledata, pickle_out)

In [1]:
#A function to clean and process berth data
def Berth_data_processing(Dayinput,Berthpath,Berthfilename,AreacodeA,AreacodeB):
    #Read in the data
    data_folder = Berthpath
    filename = data_folder+Berthfilename
    Berthdata=pd.read_csv(filename,names=['tag'])
    #Split the data into dataframe columns
    Berthdata['UTC']=Berthdata['tag'].str.slice(15,21)
    Berthdata['tag']=Berthdata['tag'].str.slice(61,77)
    Berthdata['Area']=Berthdata['tag'].str.slice(0,2)
    Berthdata['msg_type']=Berthdata['tag'].str.slice(2,4)
    Berthdata['From'] = Berthdata['tag'].str.slice(4,8)
    Berthdata['To'] = Berthdata['tag'].str.slice(8,12)
    Berthdata['Headcode'] = Berthdata['tag'].str.slice(12,16)
    del Berthdata['tag']
    #Converts the UTC column into a datetime object 
    Berthdata['RealUTC']=pd.to_datetime(Dayinput+Berthdata['UTC'],format='%Y%m%d%H%M%S')
    for i in range(0,len(Berthdata)):
        if int(Berthdata['UTC'][i])<40000:
            Berthdata.at[i,'RealUTC']=Berthdata['RealUTC'][i]+timedelta(days=1)
    #Filter the data futher selecting area code and message type
    #also retaining only the passenger trains
    BerthdataA=Berthdata[Berthdata['Area']==AreacodeA]
    #BerthdataB=Berthdata[Berthdata['Area']==AreacodeB]
    Berthdata=BerthdataA.copy()#.append(BerthdataB)
    Train=Berthdata[Berthdata['msg_type']=='CA']
    #pattern =r'[1-2][A-Z][0-9][0-9]'
    #Train=Train[Train['Headcode'].str.match(pattern)].reset_index(drop=True)
    #Example code for using pickles to save dataframes
    #pickle_out=open("filename.pickle","wb")
    #pickle.dump(Train, pickle_out)
    return Train

In [16]:
Berthdata=Berth_data_processing("20191005","/home/mathsys1/Documents/Steve/","NewTD_20191005.txt","UR","U2")
Berthdata

,UTC,Area,msg_type,From,To,Headcode,RealUTC
161,040009,UR,CA,0611,0911,4L80,2019-10-05 04:00:09
1013,040048,UR,CA,0911,0529,4L80,2019-10-05 04:00:48
1486,040112,UR,CA,0610,0604,4M83,2019-10-05 04:01:12
1788,040124,UR,CA,1053,R354,5B01,2019-10-05 04:01:24
1983,040133,UR,CA,0529,0177,4L80,2019-10-05 04:01:33
...,...,...,...,...,...,...,...
5961541,014407,UR,CA,0353,R354,2B78,2019-10-06 01:44:07
5966582,014931,UR,CA,B592,A592,5S33,2019-10-06 01:49:31
5967789,015052,UR,CA,A592,SBCE,5S33,2019-10-06 01:50:52
6025662,030012,UR,CA,C354,A354,5S35,2019-10-06 03:00:12


In [17]:
Berthdata=Berthdata.reset_index(drop=True)

In [18]:
pickle_out=open("2019-10-05Berth.pickle","wb")
pickle.dump(Berthdata, pickle_out)

In [10]:
#A function to read in a pickle data object 
def Pickle_read(pathtopickle):
    #Read in the pickle
    filename=pathtopickle
    pickle_in=open(filename,"rb")
    Train=pickle.load(pickle_in)
    return Train

In [ ]:
Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle")

In [ ]:
#A function to calculate SRTs(Sectional running times) for the
#berth data that is imported via pickles from the Berth_data_processing
#function. The input is required to be in array format
def SRT_calculation(picklepathsarray):
    Shift=pd.DataFrame(columns=['UTC','Area','msg_type','From','To','Headcode','RealUTC','prev_from','prev_to','prev_RealUTC','prev_headcode'])
    for i in range(0,len(picklepathsarray)):
        Train=Pickle_read(picklepathsarray[i])
        pattern =r'[2][A-Z][0-9][0-9]'
        Train=Train[Train['Headcode'].str.match(pattern)]
        grouped=Train.groupby('Headcode')
        for code,group in grouped:
            g=group.copy().sort_values(by='RealUTC',ascending=True)
            g['prev_from']=g['From'].shift()
            g['prev_to']=g['To'].shift()
            g['prev_RealUTC']=g['RealUTC'].shift()
            g['prev_headcode']=g['Headcode'].shift()
            g=g.dropna()
            Shift=Shift.append(g,sort=True)
    Shift=Shift.reset_index(drop=True)
    Shift['ID']=Shift['Area']+Shift['From']+Shift['To']
    Shift['Time']=Shift['RealUTC']-Shift['prev_RealUTC']
    #pattern =r'[0-9][0-9][0-9][0-9]'
    #data = Shift[Shift['To'].str.match(pattern)]
    #data = data[data['From'].str.match(pattern)]
    #data = data[data['prev_from'].str.match(pattern)]
    #data = data[data['prev_to'].str.match(pattern)]
    SRT=pd.DataFrame(columns=['Area','From','To','Headcode','UTC','msg_type','prev_from','prev_to','prev_RealUTC','Median','STD','Mean'])
    grouped=Shift.groupby('ID')
    i=0
    for code,group in grouped:
        g=group.copy().reset_index(drop=True)
        SRT=SRT.append(g.iloc[0]).reset_index(drop=True)
        SRT.at[i,'Median']=g['Time'].median()
        SRT.at[i,'STD']=g['Time'].std()
        SRT.at[i,'Mean']=g['Time'].mean()
        i=i+1
    SRT=SRT.reset_index(drop=True)
    #SRT['STD/Mean']=SRT['STD']/SRT['Mean']
    #SRT['seconds']=SRT['Time']/np.timedelta64(1,'s')
    return SRT,Shift

In [ ]:
SRT,Shift=SRT_calculation(["/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-02Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-03Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-04Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-05Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-06Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-09Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-10Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-11Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-12Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-13Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-16Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-17Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-18Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-19Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-20Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-23Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-24Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-25Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-26Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-27Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-09-30Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-02Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-03Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-04Berth.pickle"])
SRT

In [ ]:
pickle_out=open("SRT2.pickle","wb")
pickle.dump(SRT, pickle_out)

In [ ]:
dayta=Shift[Shift['ID']=='UR01120108'].reset_index(drop=True)
dayta['Time'].astype('timedelta64[s]').plot.hist(bins=20)
dayta

In [ ]:
SRT[SRT['ID']=='UR01120108']

In [ ]:
#pickle_out=open("SRT.pickle","wb")
#pickle.dump(SRT, pickle_out)

In [ ]:
#This function takes in all the previously calculated berth and timetable
#data along with the STANOX-berth-TIPLOC mapping and filters for
#a single line 
def Further_timetable_processing(Timetablepickle,STANberthpickle,Areacode):
    Timetable=Pickle_read(Timetablepickle)
    STANberth=Pickle_read(STANberthpickle)
    #Remove freight trains
    pattern =r'[1-2][A-Z][0-9][0-9]'
    Timetable=Timetable[Timetable['Headcode'].str.match(pattern)].reset_index(drop=True)
    
    #Only observe TIPLOCs from the chosen signalling area
    STANberth1=STANberth[STANberth['TD_ID']==Areacode]
    a=STANberth1['TIPLOC'].unique()
    a=a[a!='NaN']
    for i in range(0,len(Timetable)):
        if np.isin(Timetable['Location'][i],a)==False:
            Timetable=Timetable.drop(i)
    Timetable=Timetable.reset_index(drop=True)
    #Use the SchedPass column to fill the missing values in the SchedArr
    #and SchedDept columns
    for i in range(0,len(Timetable)):
        if Timetable['SchedArr'][i]==' ':
            Timetable.at[i,'SchedArr']=Timetable['SchedDept'][i]
        if Timetable['SchedArr'][i]=='     ':
            Timetable.at[i,'SchedArr']=Timetable['SchedPass'][i]
        if Timetable['SchedDept'][i]==' ':
            Timetable.at[i,'SchedDept']=Timetable['SchedArr'][i]
        if Timetable['SchedDept'][i]=='     ':
            Timetable.at[i,'SchedDept']=Timetable['SchedPass'][i]
    #Converts the SchedArr and SchedDept into more useful datetime objects
    
    Timetable['ModTime']=Timetable['SchedArr'].str.slice(0,4)
    Timetable['Half']=Timetable['SchedArr'].str.slice(4,5)
    Timetable['Time']=pd.to_datetime('20191001'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
    Timetable['ArrTime']=Timetable['Time']
    for i in range(0,len(Timetable['Half'])):
        if Timetable['Half'][i]=='H':
            Timetable.at[i,'ArrTime']=Timetable['Time'][i]+timedelta(seconds=30)
    Timetable['ModTime']=Timetable['SchedDept'].str.slice(0,4)
    Timetable['Half']=Timetable['SchedDept'].str.slice(4,5)
    Timetable['Time']=pd.to_datetime('20191001'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
    Timetable['DeptTime']=Timetable['Time']
    for i in range(0,len(Timetable['Half'])):
        if Timetable['Half'][i]=='H':
            Timetable.at[i,'DeptTime']=Timetable['Time'][i]+timedelta(seconds=30)
    return Timetable

In [ ]:
a=Further_timetable_processing("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
a

In [ ]:
#A function that calculates the median travel time between TIPLOCs
#for a chosen signalling area
def Sched_TIPLOC_tt(Timetablepickle,STANberthpickle,Areacode):
    Timetable=Further_timetable_processing(Timetablepickle,STANberthpickle,Areacode)
    #Shift the dataframe columns allowing direct comparisons in each row
    Shifty=pd.DataFrame()
    grouped=Timetable.groupby('TrainUID')
    for code,group in grouped:
        g=group.copy()
        g['prev_TIPLOC']=g['Location'].shift()
        g['prev_DeptTime']=g['DeptTime'].shift()
        Shifty=Shifty.append(g)
    Shifty=Shifty.reset_index(drop=True)
    #Calculate travel time
    Shifty['TravelTime']=Shifty['ArrTime']-Shifty['prev_DeptTime']
    Shifty=Shifty.dropna().reset_index(drop=True)
    Shifty=Shifty[Shifty['Location']!=Shifty['prev_TIPLOC']].reset_index(drop=True)
    Shifty['ID']=Shifty['prev_TIPLOC']+':'+Shifty['Location']
    #Aggregate the travel times for each TIPLOC pair
    SchedConnection=pd.DataFrame(columns=['prev_TIPLOC','Location','TravelTime','ID'])
    grouped=Shifty.groupby('ID')
    for code,group in grouped:
        g=group.copy().reset_index(drop=True)
        g.at[0,'TravelTime']=g['TravelTime'].median()
        SchedConnection=SchedConnection.append(g.loc[0]).reset_index(drop=True)
    SchedConnection['Seconds']=SchedConnection['TravelTime']/np.timedelta64(1,'s')
    #Clean up the dataframe
    del SchedConnection['Activity']
    del SchedConnection['ArrTime']
    del SchedConnection['Berth']
    del SchedConnection['Offset']
    del SchedConnection['Date']
    del SchedConnection['Dept']
    del SchedConnection['DeptTime']
    del SchedConnection['Engallow']
    del SchedConnection['Half']
    del SchedConnection['Line']
    del SchedConnection['Headcode']
    del SchedConnection['ModTime']
    del SchedConnection['Msg_type']
    del SchedConnection['Path']
    del SchedConnection['Pathallow']
    del SchedConnection['Platform']
    del SchedConnection['Perfallow']
    del SchedConnection['PubArr']
    del SchedConnection['PubDept']
    del SchedConnection['SchedArr']
    del SchedConnection['SchedDept']
    del SchedConnection['SchedPass']
    del SchedConnection['Time']
    del SchedConnection['TrainUID']
    del SchedConnection['prev_DeptTime']
    return SchedConnection

In [ ]:
SchedConnection=Sched_TIPLOC_tt("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
SchedConnection

In [8]:
#A function that adds the timetable to the berth movements
def Berth_movements_appender(Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    #Timetable=Further_timetable_processing(Timetablepickle,STANberthpickle,Areacode)
    Timetable=Pickle_read(Timetablepickle)
    Train=Pickle_read(Berthpickle)
    Train['TIPLOC']=Train['Area']
    Train['Offset']=Train['Area']
    Train['Sched']=Train['Area']
    Headcode=Train['Headcode'].unique()
    Headcode2=Timetable['Headcode'].unique()
    Df=pd.DataFrame(columns=Train.columns)
    #Create a set of headcodes that are present in both the timetable
    #and the berth movements
    Headmerge=np.empty(0)
    k=0
    for i in range(0,len(Headcode)):
        for j in range(0,len(Headcode2)):
            if Headcode[i]==Headcode2[j]:
                k=k+1
                Headmerge=np.append(Headmerge,Headcode[i])
    #Iterates through all the headcodes and connects the berth movements
    # to the timetable
    for l in range(0,len(Headmerge)):
        PrepTrain=Train[Train['Headcode']==Headmerge[l]].reset_index(drop=True)
        PrepSched=Timetable[Timetable['Headcode']==Headmerge[l]].reset_index(drop=True)
        for i in range(0,len(PrepTrain['TIPLOC'])):
            for j in range(0,len(PrepSched['Location'])):
                for k in range(0,len(PrepSched['Berth'][j])):
                    if PrepTrain['From'][i]==PrepSched['Berth'][j][k][0] and PrepTrain['To'][i]==PrepSched['Berth'][j][k][1]:
                        PrepTrain.at[i,'TIPLOC']=PrepSched['Location'][j]
                        PrepTrain.at[i,'Offset']=int(PrepSched['Offset'][j][k])
                        PrepTrain.at[i,'Sched']=PrepSched['Time'][j]
        Df=Df.append(PrepTrain)
    Df=Df.reset_index(drop=True)
    Df['Delay']=Df['Area']
    for i in range(0,len(Df['Offset'])):
        if Df['Offset'][i]=='UR' or Df['Offset'][i]=='U2':
            Df.at[i,'Offset']=0
    Df['NewUTC']=Df['RealUTC']
    for i in range(0,len(Df['NewUTC'])):
        Df.at[i,'AdjUTC']=Df['NewUTC'][i]+timedelta(seconds=int(Df['Offset'][i]))
        if Df['Sched'][i]=='UR' or Df['Sched'][i]=='U2':
            Df.at[i,'Sched']=Df['AdjUTC'][i]
        Df.at[i,'Delay']=Df['AdjUTC'][i]-Df['Sched'][i]
    del Df['RealUTC']
    del Df['msg_type']
    del Df['UTC']
    return Df

In [13]:
Df=Berth_movements_appender("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
Df

,Area,From,To,Headcode,TIPLOC,Offset,Sched,Delay,NewUTC,AdjUTC
0,UR,0667,0669,4L80,UR,0,2019-10-01 04:00:33,0 days 00:00:00,2019-10-01 04:00:33,2019-10-01 04:00:33
1,UR,0669,0671,4L80,UR,0,2019-10-01 04:01:15,0 days 00:00:00,2019-10-01 04:01:15,2019-10-01 04:01:15
2,UR,0671,0673,4L80,GRAYS,51,2019-10-01 04:09:30,-1 days +23:53:28,2019-10-01 04:02:07,2019-10-01 04:02:58
3,UR,0673,0675,4L80,GRAYS,-10,2019-10-01 04:09:30,-1 days +23:53:08,2019-10-01 04:02:48,2019-10-01 04:02:38
4,UR,0885,LG11,4L80,UR,0,2019-10-01 04:18:36,0 days 00:00:00,2019-10-01 04:18:36,2019-10-01 04:18:36
...,...,...,...,...,...,...,...,...,...,...
31451,UR,0610,0604,4M83,BARKING,0,2019-10-01 03:35:00,0 days 23:49:07,2019-10-02 03:24:07,2019-10-02 03:24:07
31452,UR,0604,L924,4M83,UR,0,2019-10-02 03:25:12,0 days 00:00:00,2019-10-02 03:25:12,2019-10-02 03:25:12
31453,UR,L924,L920,4M83,UR,0,2019-10-02 03:26:09,0 days 00:00:00,2019-10-02 03:26:09,2019-10-02 03:26:09
31454,UR,B920,LS21,4M83,UR,0,2019-10-02 03:26:47,0 days 00:00:00,2019-10-02 03:26:47,2019-10-02 03:26:47


In [14]:
pickle_out=open("20191001Df.pickle","wb")
pickle.dump(Df, pickle_out)

In [ ]:
AltTimetable=pd.DataFrame()
grouped=PrepSched.groupby('TrainUID')
for code,group in grouped:
    g=group.copy().reset_index(drop=True)
    g=g.sort_values(by=['Location'])
    for i in range(0,len(g)-1):
        if g['Location'][i]==g['Location'][i+1]:
            if g['STP'][i]=='O':
                g.drop([i+1])
                g=g.reset_index(drop=True)
                display(g)
            elif g['STP'][i+1]=='O':
                g.drop([i])
                g=g.reset_index(drop=True)
                display(g)
    #if len(g)>=1:
    #    AltTimetable=AltTimetable.append(g).reset_index(drop=True)
#AltTimetable

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
PrepSched.sort_values(by=['Location'])

In [ ]:
pickle_out=open("20190930Df.pickle","wb")
pickle.dump(Df, pickle_out)
#Dfa=Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/20190901Df.pickle")
#Dfa

In [ ]:
#Calculates a measure of how delayed a train is
def Delay_calculation(Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    Df=Berth_movements_appender(Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    Df['Delay']=Df['Area']
    for i in range(0,len(Df['Offset'])):
        if Df['Offset'][i]=='UR':
            Df.at[i,'Offset']=0
    Df['NewUTC']=Df['RealUTC']
    for i in range(0,len(Df['NewUTC'])):
        Df.at[i,'AdjUTC']=Df['NewUTC'][i]+timedelta(seconds=Df['Offset'][i])
        if Df['Sched'][i]=='UR':
            Df.at[i,'Sched']=Df['AdjUTC'][i]
        Df.at[i,'Delay']=Df['AdjUTC'][i]-Df['Sched'][i]
    del Df['RealUTC']
    del Df['msg_type']
    del Df['UTC']
    return Df

In [ ]:
Df=Delay_calculation("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
Df

In [ ]:
#Calculates the travel time between TIPLOCs from detection to detection
def Actual_TIPLOC_tt(Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    Df=Delay_calculation(Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    AltDf=Df[Df['TIPLOC']!='UR']
    #Calculate travel times between TIPLOCs
    Shift=pd.DataFrame(columns=['Area','From','To','Headcode','UTC','msg_type','prev_TIPLOC','prev_AdjUTC','prev_NewUTC'])
    grouped=AltDf.groupby('Headcode')
    for code,group in grouped:
        g=group.copy()
        g['prev_TIPLOC']=g['TIPLOC'].shift()
        g['prev_AdjUTC']=g['AdjUTC'].shift()
        g['prev_NewUTC']=g['NewUTC'].shift()
        Shift=Shift.append(g)
    Shift=Shift.reset_index(drop=True)
    Shift['TravelTime']=Shift['NewUTC']-Shift['prev_NewUTC']
    altdf=pd.DataFrame(columns=['prev_TIPLOC','TIPLOC','TravelTime'])
    altdf['TIPLOC']=Shift['TIPLOC']
    altdf['prev_TIPLOC']=Shift['prev_TIPLOC']
    altdf['TravelTime']=Shift['TravelTime']
    altdf=altdf[altdf['prev_TIPLOC']!=altdf['TIPLOC']].reset_index(drop=True)
    altdf['Connection']=altdf['prev_TIPLOC']+':'+altdf['TIPLOC']
    #Create the new dataframe to store these new values
    Connection=pd.DataFrame(columns=['prev_TIPLOC','TIPLOC','TravelTime','Connection'])
    grouped=altdf.groupby('Connection')
    for code,group in grouped:
        g=group.copy().reset_index(drop=True)
        g.at[0,'TravelTime']=g['TravelTime'].mean()
        Connection=Connection.append(g.loc[0])
    Connection=Connection.reset_index(drop=True)
    Connection['seconds']=Connection['TravelTime']/np.timedelta64(1,'s')
    return Connection

In [ ]:
Connection=Actual_TIPLOC_tt("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
Connection

In [ ]:
#Calculate predicted arrival time at the next TIPLOC predicting ahead
#based on the median SRTs between TIPLOCs
def Predict_future_arrival(picklepathsarray,Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    #Read in and process data
    BerthSRT=SRT_calculation(picklepathsarray)
    Df=Delay_calculation(Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    del BerthSRT['Headcode']
    del BerthSRT['msg_type']
    del BerthSRT['UTC']
    del BerthSRT['RealUTC']
    del BerthSRT['prev_RealUTC']
    del BerthSRT['prev_from']
    del BerthSRT['prev_to']
    #Shift the data to allow travel times to be calculated
    Shifted=pd.DataFrame(columns=['Area','From','To','Headcode','TIPLOC','Offset','Sched','Delay','NewUTC','AdjUTC'])
    grouped=Df.groupby('Headcode')
    for code,group in grouped:
        g=group.copy()
        g['prev_AdjUTC']=g['AdjUTC'].shift()
        Shifted=Shifted.append(g)
    Shifted=Shifted.reset_index(drop=True)
    Shifted['TravelTime']=Shifted['AdjUTC']-Shifted['prev_AdjUTC']
    Shifted['ID']=Shifted['Area']+Shifted['From']+Shifted['To']
    #Add the median SRT travel time
    Shifted['MedTT']=Shifted['TravelTime']
    for i in range(0,len(Shifted)):
        for j in range(0,len(BerthSRT)):
            if Shifted['ID'][i]==BerthSRT['ID'][j]:
                Shifted.at[i,'MedTT']=BerthSRT['Median'][j]
                break
    #Calculate the difference in the travel times allowing use to
    #say if the train lost or gained time in that berth section
    Shifted['TTdiff']=Shifted['TravelTime']-Shifted['MedTT']
    for i in range(1,len(Shifted)):
        if Shifted['TIPLOC'][i]=='UR':
            Shifted.at[i,'Delay']=Shifted['Delay'][i-1]+Shifted['TTdiff'][i]
    Shifted=Shifted.dropna().reset_index(drop=True)
    #Project ahead to calculate the estimated arrival time at the next TIPLOC
    a=np.timedelta64(0,'s')
    b=1
    Shifted['PredArr']=Shifted['TIPLOC']
    for i in range(0,len(Shifted)-1):
        if Shifted['TIPLOC'][i]!='UR':
            while Shifted['TIPLOC'][i+b]=='UR':
                if Shifted['TIPLOC'][i+b]=='UR':
                    a=a+Shifted['MedTT'][i+b]
                    b=b+1
            a=a+Shifted['MedTT'][i+b]
            Shifted.at[i,'PredArr']=Shifted['AdjUTC'][i]+a
            a=np.timedelta64(0,'s')
        b=1
    return Shifted

In [ ]:
Shifted=Predict_future_arrival(["/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-02Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-03Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-04Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-05Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-06Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-07Berth.pickle"],"/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
Shifted

In [ ]:
#This function calculates the travel time between TIPLOCs by adding
#up the median travel times for the intermediate berths
def Berth_level_tt(picklepathsarray,Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    #This code uses the previously calulated dataframe from the Predict_future_arrival
    #function so runs it to get the dataframe
    Shifted=Predict_future_arrival(picklepathsarray,Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    #Prepare various dataframes and counting variables
    TIPLOCtt=pd.DataFrame(columns=['From','To','tt'])
    a=np.timedelta64(0,'s')
    b=1
    c=0
    #Run an adapted version of the previous function to count berth times
    grouped=Shifted.groupby('Headcode')
    for code,group in grouped:
        g=group.copy().reset_index(drop=True)
        if g['TIPLOC'][len(g)-1]=='UR':
            g.at[len(g),'TIPLOC']='U2'
        for i in range(0,len(g)-1):
            if g['TIPLOC'][i]!='UR':
                TIPLOCtt.at[c,'From']=g['TIPLOC'][i]
                while g['TIPLOC'][i+b]=='UR':
                    if g['TIPLOC'][i+b]=='UR':
                        a=a+g['MedTT'][i+b]
                        b=b+1
                a=a+g['MedTT'][i+b]
                TIPLOCtt.at[c,'To']=g['TIPLOC'][i+b]
                TIPLOCtt.at[c,'tt']=a
                c=c+1
                a=np.timedelta64(0,'s')
            b=1
    TIPLOCtt=TIPLOCtt[TIPLOCtt['From']!=TIPLOCtt['To']].reset_index(drop=True)
    TIPLOCtt=TIPLOCtt[TIPLOCtt['To']!='U2'].reset_index(drop=True)
    TIPLOCtt['Connection']=TIPLOCtt['From']+':'+TIPLOCtt['To']
    TIPLOCtt['seconds']=TIPLOCtt['Connection']
    for i in range(0,len(TIPLOCtt)):
        TIPLOCtt.at[i,'seconds']=TIPLOCtt['tt'][i]/np.timedelta64(1,'s')
    AltConnection=pd.DataFrame(columns=['From','To','tt','Connection'])
    grouped=TIPLOCtt.groupby('Connection')
    for code,group in grouped:
        g=group.copy().reset_index(drop=True)
        g.at[0,'seconds']=g['seconds'].mean()
        AltConnection=AltConnection.append(g.loc[0]).reset_index(drop=True)
    return AltConnection

In [ ]:
AltConnection=Berth_level_tt(["/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-02Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-03Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-04Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-05Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-06Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-07Berth.pickle"],"/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
AltConnection

In [ ]:
def Compare_Connection(picklepathsarray,Berthpickle,Timetablepickle,STANberthpickle,Areacode):
    Connection=Actual_TIPLOC_tt(Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    SchedConnection=Sched_TIPLOC_tt(Timetablepickle,STANberthpickle,Areacode)
    AltConnection=Berth_level_tt(picklepathsarray,Berthpickle,Timetablepickle,STANberthpickle,Areacode)
    SumConnection=pd.DataFrame(columns=['From','To','ID'])
    a=Connection['Connection']
    a=a.append(SchedConnection['ID'])
    a=a.append(AltConnection['Connection'])
    a=a.unique()
    SumConnection['ID']=a
    SumConnection['From']=SumConnection['ID'].str.slice(0,7)
    SumConnection['To']=SumConnection['ID'].str.slice(7,14)
    SumConnection['TIPLOCseconds']=SumConnection['From']
    SumConnection['Berthseconds']=SumConnection['From']
    SumConnection['Schedseconds']=SumConnection['From']
    for i in range(0,len(SumConnection)):
        for j in range(0,len(Connection)):
            if SumConnection['ID'][i]==Connection['Connection'][j]:
                SumConnection.at[i,'TIPLOCseconds']=Connection['seconds'][j]
        for k in range(0,len(AltConnection)):
            if SumConnection['ID'][i]==AltConnection['Connection'][k]:
                SumConnection.at[i,'Berthseconds']=AltConnection['seconds'][k]
        for l in range(0,len(SchedConnection)):
            if SumConnection['ID'][i]==SchedConnection['ID'][l]:
                SumConnection.at[i,'Schedseconds']=SchedConnection['Seconds'][l]
    for i in range(0,len(SumConnection)):
        x=SumConnection['ID'][i].split(":")
        SumConnection.at[i,'From']=x[0]
        SumConnection.at[i,'To']=x[1]
    return SumConnection

In [ ]:
SumConnection=Compare_Connection(["/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-02Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-03Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-04Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-05Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-06Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-07Berth.pickle"],"/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Berth.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Data_pickles/2019-10-01Timetable.pickle","/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle","UR")
SumConnection